In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os.path import join
import matplotlib.pyplot as plt

### Check that the donor details is in alignment with the files we got

In [61]:
import pandas as pd
from os import listdir
from os.path import join, exists

In [2]:
df = pd.read_csv('data/donor_details.tsv', sep='\t')

In [10]:
for ri, r in df.iterrows():
    path = join('/storage/groups/ml01/datasets/projects/20210318_retinal_data_integration_ignacio.ibarra_malte.luecken', r['file'])
    if not exists(path):
        print(exists(path), path)


In [12]:
import scanpy as sc

In [13]:
# this input contains the raw counts for all cells, and needs to be updated in the raw sesion of the object.
ad_raw = '/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/input/input_all_cells.h5ad'
print('reading...')
ad_raw = sc.read_h5ad(ad_raw)
print('done...')

reading...
done...


In [45]:
ad2.obs['scpred_prediction'].value_counts()

Microglia     49
unassigned    15
Astrocyte     15
RPE            5
MG             1
Name: scpred_prediction, dtype: int64

In [23]:
filename_by_donor = df.set_index('donor')['file'].to_dict()

In [48]:
# missing files are containing less than 50 cells per cell type. This was an arbitrary definition to consider these.
for f in (set(df['file']) - set('./' + ad_raw.obs['dataset'].astype(str) + '/' + ad_raw.obs['filename'].astype(str) + '.h5ad')):
    print(f)
    ad2 = sc.read_h5ad(join('/storage/groups/ml01/datasets/projects/20210318_retinal_data_integration_ignacio.ibarra_malte.luecken', f))
    print(ad2.obs['scpred_prediction'].value_counts())

./Roska/R-00646_04_Fovea_ChoroidRPE_Right.h5ad
Microglia     74
unassigned    15
Astrocyte      9
MG             1
RPE            1
Name: scpred_prediction, dtype: int64
./Sanes/H3FoveaS3.h5ad
HC     9
Rod    5
MG     5
BC     3
RGC    1
Name: scpred_prediction, dtype: int64
./Roska/R-00646_04_Fovea_ChoroidRPE_Left.h5ad
Microglia     49
unassigned    15
Astrocyte     15
RPE            5
MG             1
Name: scpred_prediction, dtype: int64


In [53]:
(ad_raw.obs['filename'] == 'H3FoveaS3').value_counts()

False    2159366
Name: filename, dtype: int64

In [29]:
len(ad_raw.obs['donor'].map(filename_by_donor).value_counts())

50

In [20]:
ad_raw.obs['filename'].value_counts()

10x3_Lobe_19_D011_NeuN                   227481
10x3_Lobe_19_D006_NeuN                   195551
10x3_Lobe_19_D019_NeuN                    97844
10x3_Lobe_D013_13_NeuN                    85084
10x3_Lobe_19_D003_NeuN                    83556
                                          ...  
GSM3745994                                  347
GSM3745993                                  339
R-00646_03_Periphery_ChoroidRPE_Right       288
GSM4081524                                  273
R-00646_01_Periphery_ChoroidRPE_Right       250
Name: filename, Length: 142, dtype: int64

In [56]:
# Fix export PDF as text issue
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

fpath = join(plt.rcParams["datapath"], "fonts/ttf/arial.ttf")
from matplotlib import font_manager as fm, rcParams
prop = fm.FontProperties(fname=fpath)
rcParams["font.family"] = "arial"

import os
import scanpy as sc
from os.path import join, exists
from os import listdir
import anndata
import scipy
import numpy as np
import sys


import seaborn as sns
sns.set()
from matplotlib import rcParams
import matplotlib.pyplot as plt

from utils import *

# convert counts into float32
# Convenience method for computing the size of objects
def print_size_in_MB(x):
    return '{:.3} MB'.format(x.__sizeof__()/1e6)

def print_size_in_MB_sparse_matrix(a):
    # a = scipy.sparse.csr_matrix(np.random.randint(10, size=(40, 3)))
    # x = a.data.nbytes + a.indptr.nbytes + a.indices.nbytes
    size = a.data.size/(1024**2)
    return '{:.3} MB'.format(size)

import warnings
warnings.filterwarnings("ignore")

NameError: name 'plt' is not defined

In [57]:
ls -ltrh ../../data/integration_march_2021/input/

total 32G
drwxr-xr-x. 2 ignacio.ibarra OG-ICB-User   11 Apr 30 14:31 bydataset_500/
drwxr-xr-x. 2 ignacio.ibarra OG-ICB-User   11 Apr 30 14:32 bydataset_1000/
drwxr-xr-x. 2 ignacio.ibarra OG-ICB-User   11 Apr 30 14:33 bydataset/
drwxr-xr-x. 2 ignacio.ibarra OG-ICB-User   11 May 12 20:56 bydataset_250/
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User 152M Jun  3 09:50 input_250_cells_part3.h5ad
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User 208M Jun  3 09:50 input_250_cells.h5ad
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User 302M Jun  3 09:51 input_500_cells_part3.h5ad
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User 441M Jun  3 09:51 input_500_cells.h5ad
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User 599M Jun  3 09:52 input_1000_cells_part3.h5ad
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User 874M Jun  3 09:53 input_1000_cells.h5ad
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User  17G Jun  3 10:23 input_all_cells_part3.h5ad
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User  12G Jun  3 10:32 input_all_cells.h5ad


In [58]:
input_dir = '../../data/integration_march_2021/input'

In [62]:
ad_by_f = {}
for f in listdir(input_dir):
    if not f.endswith('h5ad') or 'part' in f:
        continue
    print(f)
    
    p = join(input_dir, f)
    ad = sc.read_h5ad(p)
    ad_by_f[f] = ad
    print('done...')
    print(ad)    

input_500_cells.h5ad
done...
AnnData object with n_obs × n_vars = 69631 × 3841
    obs: 'RNA_snn_res.0.8', 'batch', 'dataset', 'filename', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'scpred_AC', 'scpred_Astrocyte', 'scpred_BC', 'scpred_Cone', 'scpred_HC', 'scpred_MG', 'scpred_Microglia', 'scpred_RGC', 'scpred_RPE', 'scpred_Rod', 'scpred_max', 'scpred_prediction', 'seurat_clusters', 'size_factors', 'cell.type', 'batch.merged', 'donor'
    layers: 'counts'
input_1000_cells.h5ad
done...
AnnData object with n_obs × n_vars = 134503 × 4063
    obs: 'RNA_snn_res.0.8', 'batch', 'dataset', 'filename', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'scpred_AC', 'scpred_Astrocyte', 'scpred_BC', 'scpred_Cone', 'scpred_HC', 'scpred_MG', 'scpred_Microglia', 'scpred_RGC', 'scpred_RPE', 'scpred_Rod', 'scpred_max', 'scpred_prediction', 'seurat_clusters', 'size_factors', 'cell.type', 'batch.merged', 'donor'
    layers: 'counts'
input_250_cells.h5ad
done...
AnnData object with n_obs × n_vars = 35492 × 3282

In [63]:
ad.obs

,RNA_snn_res.0.8,batch,dataset,filename,nCount_RNA,nFeature_RNA,orig.ident,scpred_AC,scpred_Astrocyte,scpred_BC,...,scpred_RGC,scpred_RPE,scpred_Rod,scpred_max,scpred_prediction,seurat_clusters,size_factors,cell.type,batch.merged,donor
AAACCTGCACCCAGTG-1-0:Wong:0,3,0,Wong,Retina_3B,4463.0,2034,0,1.376991e-07,0.000028,0.000057,...,0.000085,0.000051,0.999978,0.999978,Rod,3,1.023493,Rod,140,Wong_donor3
AAACCTGTCTATCCTA-1-0:Wong:0,9,0,Wong,Retina_3B,2075.0,1217,0,4.611996e-03,0.000073,0.000165,...,0.000221,0.000082,0.000176,0.999648,HC,9,0.609796,HC,140,Wong_donor3
AAACGGGAGCAGACTG-1-0:Wong:0,0,0,Wong,Retina_3B,1276.0,767,0,4.067842e-07,0.000014,0.000087,...,0.000044,0.000047,0.999962,0.999962,Rod,0,0.305145,Rod,140,Wong_donor3
AAACGGGTCCCGGATG-1-0:Wong:0,0,0,Wong,Retina_3B,1267.0,738,0,8.419816e-07,0.000012,0.000122,...,0.000069,0.000053,0.999916,0.999916,Rod,0,0.329595,Rod,140,Wong_donor3
AAAGATGCAAATCCGT-1-0:Wong:0,1,0,Wong,Retina_3B,1213.0,806,0,4.195110e-07,0.000015,0.000059,...,0.000063,0.000050,0.999963,0.999963,Rod,1,0.312810,Rod,140,Wong_donor3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTTCGTGA-1-39:Chen_a:39,1,39,Chen_a,10x_Lobe_D017_18_13_Nu,3163.0,1671,0,8.383034e-07,0.000036,0.000028,...,0.000096,0.000058,0.999829,0.999829,Rod,1,0.543456,Rod,33,Chen_D017_13
TTTGTTGTCATGAGTC-1-39:Chen_a:39,4,39,Chen_a,10x_Lobe_D017_18_13_Nu,2776.0,1467,0,6.765336e-05,0.001598,0.000098,...,0.000103,0.000091,0.000167,0.990593,MG,4,0.410273,MG,33,Chen_D017_13
TTTGTTGTCCTGTAAG-1-39:Chen_a:39,0,39,Chen_a,10x_Lobe_D017_18_13_Nu,3552.0,1705,0,4.776147e-06,0.000022,0.000101,...,0.000061,0.000060,0.999834,0.999834,Rod,0,0.611342,Rod,33,Chen_D017_13
TTTGTTGTCGAAACAA-1-39:Chen_a:39,0,39,Chen_a,10x_Lobe_D017_18_13_Nu,4630.0,2218,0,1.915799e-07,0.000018,0.000052,...,0.000085,0.000053,0.999957,0.999957,Rod,0,0.758877,Rod,33,Chen_D017_13


In [71]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import os

In [73]:
for batch_type in ['donor', 'dataset', 'batch.merged']:
    if batch_type != 'donor':
        continue
    
    for f in ad_by_f:
        print(f)

        ad = ad_by_f[f]
        import pandas as pd

        if batch_type == 'batch.merged':
            ad.obs['batch.full'] = ad.obs['dataset'].astype(str) + '_' + ad.obs['batch'].astype(str)
        elif batch_type == 'donor':
            ad.obs['batch.full'] = ad.obs['donor'].astype(str)
        else:
            ad.obs['batch.full'] = ad.obs['dataset'].astype(str)
        df = ad.obs[['cell.type', 'batch.full']].value_counts().reset_index()
        df = df.pivot('batch.full', 'cell.type', 0)

        name = df.index
        df = df.reset_index(drop=True)
        df = pd.DataFrame(np.array(df.fillna(0)).astype(int), columns=list(df.columns), index=list(name))

        df['sum'] = df.sum(axis=1)
        df['k'] = ['_'.join(a) for a in df.index.str.split('_').str[:-1]]

        df = df.sort_values(['k','sum'], ascending=[False, False])
        del df['sum']
        del df['k']

        rcParams['figure.figsize'] = [15, 7]
        rcParams['figure.dpi'] = 120

        from itertools import cycle, islice
        import pandas, numpy as np  # I find np.random.randint to be better
        c = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', 'lightgrey']
        my_colors = list(islice(cycle(c), None, len(df.columns)))

        n_total_cells = df.sum().sum()
        n_batches = len(df.index)
        cells_by_batch = df.sum(axis=1).mean()
        for pi in [0, 1]:
            ax = plt.subplot(2, 1, pi + 1)
            df.plot(kind='bar', stacked=True, width=1.0, linewidth=0.2, logy=pi == 0,
                    color=my_colors, edgecolor='black', ax=ax)
            plt.ylabel('# cells%s' % (' (log)' if pi == 0 else ''))
            plt.xlabel('dataset batch')
            plt.xticks(fontsize=6)
            if pi == 0:
                plt.title('# cells = %s, # batches = %i, # cells by batch = %s' % ('{:,}'.format(n_total_cells), n_batches, '{:,}'.format(cells_by_batch)))
            plt.legend(fontsize=10, title='scpred', bbox_to_anchor=(1.01, 1))

        
        plt.tight_layout()

        img_path = '../../data/scIB_run/%s_%s' % (batch_type, f.replace('.h5ad', '_barplot_scpred.png'))
        plt.savefig(img_path)

        print(os.path.abspath(img_path)) # .replace('.pdf', '.png')))
        plt.close()


input_500_cells.h5ad
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/scIB_run/donor_input_500_cells_barplot_scpred.png
input_1000_cells.h5ad
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/scIB_run/donor_input_1000_cells_barplot_scpred.png
input_250_cells.h5ad
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/scIB_run/donor_input_250_cells_barplot_scpred.png
input_all_cells.h5ad
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/scIB_run/donor_input_all_cells_barplot_scpred.png


In [19]:
print(os.path.abspath(img_path))

/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/scIB_run/input_250_cells_barplot_scpred.png


In [ ]:
%matplotlib inline

In [ ]:
from PIL import Image

image = Image.open(img_path)
image.show()